In [0]:
# 틱택토 구현
import random

# 게임 상태
class State:
    # 초기화
    def __init__(self, pieces=None, enemy_pieces=None):
        # 돌 배치
        self.pieces = pieces if pieces != None else [0] * 9
        self.enemy_pieces = enemy_pieces if enemy_pieces != None else [0] * 9

    # 돌의 수 취득
    def piece_count(self, pieces):
        count = 0
        for i in pieces:
            if i == 1:
                count +=  1
        return count

    # 패배 여부 확인
    def is_lose(self):
        # 돌 3개 연결 여부
        def is_comp(x, y, dx, dy):
            for k in range(3):
                if y < 0 or 2 < y or x < 0 or 2 < x or \
                    self.enemy_pieces[x+y*3] == 0:
                    return False
                x, y = x+dx, y+dy
            return True

        # 패배 여부 확인
        if is_comp(0, 0, 1, 1) or is_comp(0, 2, 1, -1):
            return True
        for i in range(3):
            if is_comp(0, i, 1, 0) or is_comp(i, 0, 0, 1):
                return True
        return False

    # 무승부 여부 확인
    def is_draw(self):
        return self.piece_count(self.pieces) + self.piece_count(self.enemy_pieces) == 9

    # 게임 종료 여부 확인
    def is_done(self):
        return self.is_lose() or self.is_draw()

    # 다음 상태 얻기
    def next(self, action):
        pieces = self.pieces.copy()
        pieces[action] = 1
        return State(self.enemy_pieces, pieces)

    # 합법적인 수의 리스트 얻기
    def legal_actions(self):
        actions = []
        for i in range(9):
            if self.pieces[i] == 0 and self.enemy_pieces[i] == 0:
                actions.append(i)
        return actions

    # 선 수 여부 확인
    def is_first_player(self):
        return self.piece_count(self.pieces) == self.piece_count(self.enemy_pieces)

    # 문자열 표시
    def __str__(self):
        ox = ('o', 'x') if self.is_first_player() else ('x', 'o')
        str = ''
        for i in range(9):
            if self.pieces[i] == 1:
                str += ox[0]
            elif self.enemy_pieces[i] == 1:
                str += ox[1]
            else:
                str += '-'
            if i % 3 == 2:
                str += '\n'
        return str

In [0]:
# 랜덤으로 행동 선택
def random_action(state):
    legal_actions = state.legal_actions()
    return legal_actions[random.randint(0, len(legal_actions)-1)]

In [0]:
# 랜덤과 랜덤의 대전

# 상태 생성
state = State()

# 게임 종료 시까지 반복
while True:
    # 게임 종료 시
    if state.is_done():
        break;

    # 행동 얻기
    action = random_action(state)

    # 다음 상태 얻기
    state = state.next(action)

    # 문자열 표시
    print(state)
    print()

---
o--
---


---
ox-
---


---
ox-
--o


--x
ox-
--o


-ox
ox-
--o


-ox
oxx
--o


-ox
oxx
o-o


xox
oxx
o-o


xox
oxx
ooo




In [0]:
# 미니맥스법을 활용한 상태 가치 계산
def mini_max(state):
    # 패배 시, 상태 가치 -1
    if state.is_lose():
        return -1
    
    # 무승부 시, 상태 가치 0
    if state.is_draw():
        return  0

    # 합법적인 수의 상태 가치 계산
    best_score = -float('inf')
    for action in state.legal_actions():
        score = -mini_max(state.next(action))
        if score > best_score:
            best_score = score
            
    # 합법적인 수의 상태 가치값 중 최대값 선택
    return best_score

# 미니맥스법을 활용한 행동 선택
def mini_max_action(state):
    # 합법적인 수의 상태 가치 계산
    best_action = 0
    best_score = -float('inf')
    str = ['','']
    for action in state.legal_actions():
        score = -mini_max(state.next(action))
        if score > best_score:
            best_action = action
            best_score  = score
            
        str[0] = '{}{:2d},'.format(str[0], action)
        str[1] = '{}{:2d},'.format(str[1], score)
    print('action:', str[0], '\nscore: ', str[1], '\n')

    # 합법적인 수의 상태 가치의 최대값을 가진 행동 반환
    return best_action

In [0]:
# 미니맥스법과 랜덤의 대전

# 상태 생성
state = State()

# 게임 종료 시까지 반복
while True:
    # 게임 종료 시
    if state.is_done():
        break

    # 행동 얻기
    if state.is_first_player():
        action = mini_max_action(state)
    else:
        action = random_action(state)
    
    # 다음 상태 얻기
    state = state.next(action)

    # 문자열 표시
    print(state)
    print()

action:  0, 1, 2, 3, 4, 5, 6, 7, 8, 
score:   0, 0, 0, 0, 0, 0, 0, 0, 0, 

o--
---
---


o--
---
-x-


action:  1, 2, 3, 4, 5, 6, 8, 
score:   0, 1,-1, 1, 0, 1, 0, 

o-o
---
-x-


o-o
-x-
-x-


action:  1, 3, 5, 6, 8, 
score:   1,-1,-1,-1,-1, 

ooo
-x-
-x-


